# Start simple: show the class works for a small grid

In [1]:
import numpy as np
from grid import WorkJourney

In [2]:
reward_dict = {'r_time' : -1, 
               'r_pond' : -15,
               'r_croissant' : 200,
               'r_cogs' : 200,
               'r_work' : 15}
ob = WorkJourney(reward_dict, max_steps=500, max_episodes=2000, 
                 start_state=6, goal_state=35)
